In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('../')

In [ ]:
# specify parameters for interactive run
# this cell will be replaced during pipeline run with the parameters from notebook function call
boston_dataset = '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053319/boston_dataset'

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
# read inputs 
boston_df = spark.read.parquet(boston_dataset)

In [ ]:
# prepare the data and divide the boston dataset into train, eval and test subsets
from sklearn.model_selection import train_test_split

train_df, test_df = boston_df.randomSplit(weights=[0.7,0.3], seed=239)
eval_df, test_df = test_df.randomSplit(weights=[0.5,0.5], seed=239)

x_train = train_df.drop("MEDV")
y_train = train_df.select("MEDV")

x_eval = eval_df.drop("MEDV")
y_eval = eval_df.select("MEDV")

x_test = test_df.drop("MEDV")
y_test = test_df.select("MEDV")

In [ ]:
#write outputs
from sinara.substep import get_tmp_work_path
outputs = get_tmp_work_path()

x_train.write.parquet(f"{outputs}/df_X_train", mode='overwrite')
y_train.write.parquet(f"{outputs}/df_Y_train", mode='overwrite')

x_eval.write.parquet(f"{outputs}/df_X_eval", mode='overwrite')
y_eval.write.parquet(f"{outputs}/df_Y_eval", mode='overwrite')

x_test.write.parquet(f"{outputs}/df_X_test", mode='overwrite')
y_test.write.parquet(f"{outputs}/df_Y_test", mode='overwrite')

In [ ]:
#8 stop spark
SinaraSpark.stop_session()

In [ ]:
# results used by the model_train step
{
    'df_X_train': f"{outputs}/df_X_train",
    'df_Y_train': f"{outputs}/df_Y_train",
    
    'df_X_eval': f"{outputs}/df_X_eval",
    'df_Y_eval': f"{outputs}/df_Y_eval",
    
    'df_X_test': f"{outputs}/df_X_test",
    'df_Y_test': f"{outputs}/df_Y_test",
}